In [1]:
pip install pandas
import pandas as pd


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
df = pd.read_csv('Historical Lead Records.csv', encoding='latin1')

df.head()

,Title,Job Role,Job Function,Job Level
0,Manager-Cybersecurity,Information Security,IT,Manager
1,"Manager, Information Security",Information Security,IT,Manager
2,User Experience Analyst,Development,Engineering,Contributor
3,Network Specialist,Networking,IT,Contributor
4,Director of Privacy and Compliance,Information Security,IT,Director


In [ ]:
#data relableing to most frequent F/R/L
def most_frequent(x):
    if not x.empty and len(x.value_counts()) > 0:
        return x.value_counts().index[0]
    else:
        return None

grouped = df.groupby('Title').agg(most_frequent)
grouped.reset_index(inplace=True)

merged_df = pd.merge(df, grouped, on='Title', suffixes=('', '_most_frequent'))

merged_df.drop(['Job Role', 'Job Function', 'Job Level'], axis=1, inplace=True)

merged_df.rename(columns={'Job Role_most_frequent': 'Job Role', 'Job Function_most_frequent': 'Job Function', 'Job Level_most_frequent': 'Job Level'}, inplace=True)


In [2]:
#make title column lowercase
df['Title'] = df['Title'].str.lower()

df.shape[0]

865671

In [3]:
na_counts = df.isna().sum()
print(na_counts)

Title           12682
Job Role         4502
Job Function     5083
Job Level       24546
dtype: int64


In [4]:
#1616 rows are completely empty
before_dropping = df.shape[0]
empty_rows= df.isna().all(axis=1).sum()
df = df.dropna(how='all')
after_dropping = df.shape[0]


print (before_dropping)
print (empty_rows)
print (after_dropping)

865671
1616
864055


In [5]:
question_mark = df['Title'].str.contains('\?').sum()
question_mark

148

In [6]:
# #replacing all question marks with empty strings
# df['Title'] = df['Title'].str.replace('?', '')
# question_mark1 = df['Title'].str.contains('\?').sum()
# question_mark1

In [7]:
df = df.dropna(subset=['Title'])
df.shape[0]

852989

In [10]:
pip install langdetect

     -------------------------------------- 981.5/981.5 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993228 sha256=37feb88ce293ddb86268dcbe1a5534439cd421c92a656fcb4c79a8661e32bb3d
  Stored in directory: c:\users\rajat\appdata\local\pip\cache\wheels\73\b2\db\0c9b9eb7a44bf85ec0b42c06ee617d0a0de66840dc0b3248d1
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from joblib import Parallel, delayed
import numpy as np

def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

def filter_english_titles(df):
    return df[df['Title'].apply(is_english)]

num_jobs = 4
chunks = np.array_split(df, num_jobs)
df_filtered_chunks = Parallel(n_jobs=num_jobs)(
    delayed(filter_english_titles)(chunk) for chunk in chunks
)
df_filtered = pd.concat(df_filtered_chunks)



In [12]:
df_filtered.shape[0]

550254

In [13]:
df_filtered.head()

,Title,Job Role,Job Function,Job Level
0,manager-cybersecurity,Information Security,IT,Manager
1,"manager, information security",Information Security,IT,Manager
2,user experience analyst,Development,Engineering,Contributor
3,network specialist,Networking,IT,Contributor
4,director of privacy and compliance,Information Security,IT,Director


In [14]:
def replace_role(row):
    if pd.isnull(row['Job Function']): 
        return 'N/A'
    elif 'IT' not in row['Job Function']:
        return 'N/A'
    else:
        return row['Job Role']

df_filtered['Job Role'] = df_filtered.apply(replace_role, axis=1)

In [15]:
df_filtered['Job Function'] = df_filtered['Job Function'].fillna('')

non_it = df_filtered[~df_filtered['Job Function'].str.contains('IT')]
non_it.head()

,Title,Job Role,Job Function,Job Level
2,user experience analyst,N/A,Engineering,Contributor
14,"branch manager, purchasing",N/A,Purchasing,Manager
18,legislative affairs director,N/A,Legal,Director
25,associate director of software development,N/A,Engineering,Director
28,turning vision into reality,N/A,Engineering,Unknown


In [ ]:
# delete rows where a title column is empty?
#rows with question marks?

In [16]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [17]:
job_levels = df['Job Level'].value_counts()
job_functions = df['Job Function'].value_counts()

print(job_levels.head())
print(job_functions.head())

Contributor    306520
Manager        175440
Director       143138
Executive      109592
C-Level         97365
Name: Job Level, dtype: int64
IT             602041
Engineering     80296
Sales           35447
Management      32220
Unknown         21958
Name: Job Function, dtype: int64


In [18]:
pip install scikit-learn gensim

     -------------------------------------- 24.0/24.0 MB 268.9 kB/s eta 0:00:00
     ------------------------------------ 983.8/983.8 kB 566.7 kB/s eta 0:00:00
     -------------------------------------- 60.8/60.8 kB 293.9 kB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.13
    Uninstalling Cython-0.29.13:
      Successfully uninstalled Cython-0.29.13
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import gensim 
#Some more preprocessing that will remove punctuations. 
tokenised_titles = df['Title'].apply(gensim.utils.simple_preprocess)

In [23]:
tokenised_titles.head()

0                    [manager, cybersecurity]
1            [manager, information, security]
2                 [user, experience, analyst]
3                       [network, specialist]
4    [director, of, privacy, and, compliance]
Name: Title, dtype: object

In [53]:
#add the tokens as a column in df
df.insert(1, 'Title_Tokens', tokenised_titles)
df.head()

ValueError: cannot insert Title_Tokens, already exists

In [37]:
#initialize and train model
model = gensim.models.Word2Vec(window=2,min_count=1, workers=4)
model.build_vocab(df['Title_Tokens'])#this is a required step before training the model 
model.train(df['Title_Tokens'], total_examples=model.corpus_count, epochs=model.epochs) #default vector_size = 100


(6884111, 14370565)

In [31]:
model.wv.most_similar("manager")

[('director', 0.7827468514442444),
 ('mgr', 0.7687580585479736),
 ('coordinator', 0.7348541617393494),
 ('leader', 0.7106366753578186),
 ('supervisor', 0.7100642919540405),
 ('manger', 0.6971644163131714),
 ('lead', 0.6788665652275085),
 ('specialist', 0.6764187812805176),
 ('manage', 0.6743870377540588),
 ('management', 0.6501807570457458)]

In [38]:
model.wv.most_similar("analyst")

[('specialist', 0.7546834945678711),
 ('analysts', 0.7498314380645752),
 ('manager', 0.601697564125061),
 ('technician', 0.5991343855857849),
 ('coordinator', 0.5934416055679321),
 ('supervisor', 0.5820227861404419),
 ('engineer', 0.5717669129371643),
 ('administrator', 0.5643339157104492),
 ('enginner', 0.5302433967590332),
 ('analysis', 0.5218009352684021)]

In [39]:
model.wv.most_similar("network")

[('noc', 0.5734050273895264),
 ('networks', 0.5691438317298889),
 ('video', 0.5559564828872681),
 ('facilities', 0.5432752966880798),
 ('telecom', 0.5405401587486267),
 ('datacenter', 0.5379189252853394),
 ('voice', 0.5366911292076111),
 ('av', 0.5298367738723755),
 ('nodes', 0.5129046440124512),
 ('ip', 0.511024534702301)]

In [60]:
import numpy as np

#below function creates a vector for a title by adding the vector of each word in it
def get_title_vec(title_tokens):
    #create a list of vectors of all the tokens in a title
    vectors = [ model.wv[token] for token in title_tokens]
    #sum all the vectors in the list
    return np.sum(vectors, axis=0)

title_vecs = df['Title_Tokens'].map(get_title_vec)
#Add title vecs as a column in df 
df.insert(2, 'Title_Vec', title_vecs)
df.head()

,Title,Title_Tokens,Title_Vec,Job Role,Job Function,Job Level
0,manager-cybersecurity,"[manager, cybersecurity]","[-0.114685684, -0.48989522, 0.46392983, 0.0296...",Information Security,IT,Manager
1,"manager, information security","[manager, information, security]","[1.5701616, -0.77917975, 0.48444414, 1.4542677...",Information Security,IT,Manager
2,user experience analyst,"[user, experience, analyst]","[0.7436342, 2.3974192, -1.3278624, 2.3217463, ...",Development,Engineering,Contributor
3,network specialist,"[network, specialist]","[0.96099746, 0.24122985, 0.79469705, 1.1975677...",Networking,IT,Contributor
4,director of privacy and compliance,"[director, of, privacy, and, compliance]","[1.5009707, -2.0199673, 2.4246416, 1.1331086, ...",Information Security,IT,Director
